## A Brief Introduction to Multilayer Perceptions
A Perceptron is a single neuron model that was a precursor to larger neural networks. A perceptron is able to receive an single dimensional array of inputs and produce an single dimensional array of outputs that can be used for a variety of ML tasks, the most fundamental of which is classification. 

The building block for neural networks are artificial neurons. These are simple computational units that have weighted input signals and produce an output signal using an activation function. 

The weights attached to input signals can be likened to the coefficients of a regression equation. These weights represent the level of importance for each input. Consequently, changes in some inputs will have a larger effect on the output of the perceptron than others. Furthermore, each neuron will have a bias, a scalar weight that adjusts the final output of the neuron by a set amount, positive or negative.

Weights are often initialized to small random values although more complex initialization schemes can be used. Larger weights tend to increase the complexity and fragility of the model, so it is desirable to keep weights small and regularization techniques can be used to decrease the risks of overfitting. 

### Activation

The weighted inputs are summed and passed through an **activation function**, sometimes called a **transfer function**. It is called an activation function because it governs the threshold at which the neuron is "activated" and the strength of the output signal. The output of the function is limited in various ways, from application of the sigmoid function to bind values between 0 and 1, to hyperbolic tangent functions (Tanh) that binds values between -1 and 1, to ReLU (rectifier activation function) that binds values between 0 and another positive number. ReLU has been found to provide better results, generally, and this may be due to its mimicry of all-or-nothing type activation systems found in biological neurology. 

### Network of Neurons

Neurons are arranged into networks of neurons. A row of neurons is called a **layer** and one network can have multiple layers. The architecture of the neurons in the network is often called the **network topology**. 

### Input of Visible Layers

The bottom layer that takes input from a dataset is called the visible layer, because it is the exposed part of the network. Fundamentally, it is simply the series of initial inputs, where each input is the value from each column in the dataset for a given observation. 

### Hidden Layers
Layers after the input layer are called **hidden layers** because they are not directly exposed to the input. Each input is effectively transformed through a weight and bias. 

### Output Layer

The final hidden layer is called the output layer and is responsible for producing a value or vector of values that correspond to the format required for the problem. Here are some examples:
- A regression problem may have a single output neuron and the neuron may have no activation function.
- A binary classification problem may have a single output neuron and use a sigmoid activation function to output a value between 0 and 1 to represent the probability of predicting a value for the primary class. This can be turned into a crisp class value by using a threshold of 0.5 and snap values less than the threshold to 0 otherwise to 1. 
- A multiclass classification problem may have multiple neurons in the output layer, one for each class (e.g. three neurons for the three classes in the famous iris flowers classification problem). In this case, a softmax activation function may be used to output a probability of the network predicting each of the class values. Selecting the output with the highest probability can be used to produce a crisp class classification value. 

## Training Networks
Once configured, the neural network must be trained on the dataset.

### Data Preparation
- Data must be numerical
    - Categorical data can be converted via *one-hot encoding*.
    - Text data will need to be converted via TF-IDF or some other NLP encoding technique.
    - This same one hot encoding can be used on the output variable in classification problems with more than one class.
- Data must be similarly scaled
    - Min-max scaling (or any other scaling transformation) can be applied to the dataset to ensure consistency between inputs
    
### Stochastic Gradient Descent
The classical training algorithm for neural networks is stochastic gradient descent. A single observation is exposed to the network, and the output is generated (called a **forward pass**). The output is compared to the expected output and an error is calculated. The error is propagated back through the network, one layer at a time, and the weights are updated according to the amount they contributed ot the error (known as **backpropagation**). One round of updating the network for the entire training dataset is called an **epoch**. A network may be trained for any number of epochs. 

### Weight Updates
The weights can be updated from the errors calculated after each forward pass for each observation. This is called **online learning**. Alternatively, the errors can be aggregated across all results of all observations in the training set. This is called **batch learning** and is often more stable. 

Because datasets are so large, the size of the batch will be reduced before an update is completed. The amount that the weights are updated is controlled by a configuration parameter called the learning rate. This is also called the **step size** and controls the momentum at which a local minima is approached in order to reduce the risk of a convergence failure (where a local minima cannot be identified because each update overshoots a true local minima of the aggregated errors). 
- **Momentum** is a term that incorporates the properties from the previous weight update to allow the weights to continue to change in the same direction even when there is less error being calculated.
- **Learning Rate Decay** is used to decrease the learning weight over epochs to allow the network to make large changes to the weights at the beginning and smaller fine tuning changes later in the training schedule.

### Prediction
Predictions are made by providing the input to the network and performing a forward-pass allowing it to generate an ouput that you can use as a prediction. Evaluating performance on a separate out-of-sample dataset (ideally randomly split from the original full dataset available) can alert to risks of overfitting.

The network topology and the final set of weights is all that needs to persist from the research environment into the production environment to make novel predictions. 

## Developing a Simple Neural Network from the Pima Indians Onset of Diabetes Dataset

This is a standard machine learning dataset available from the UCI Machine Learning repository. It describes patient medical record data for Pima Indians and whether they had an onset of diabetes within five years. It is a binary classification problem (onset of diabetes as 1 or not as 0). Below are the attributes for the dataset:
1. Number of times pregnant.
2. Plasma glucose concentration at 2 hours in an oral glucose tolerance test.
3. Diastolic blood pressure (mm Hg).
4. Triceps skin fold thickness (mm).
5. 2-Hour serum insulin (mu U/ml).
6. Body mass index.
7. Diabetes pedigree function.
8. Age (years).
9. Class, onset of diabetes within five years.

## Loading Data

In [1]:
# Import relevant libraries
import numpy as np
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import train_test_split

# Remove warnings
import warnings
warnings.filterwarnings('ignore')

# Fix random seed for reproducibility
seed = 7
tf.random.set_seed(seed)

In [2]:
# loading the dataset
dataset = np.loadtxt('pima-indians-diabetes.csv', delimiter=',')

dataset

array([[  6.   , 148.   ,  72.   , ...,   0.627,  50.   ,   1.   ],
       [  1.   ,  85.   ,  66.   , ...,   0.351,  31.   ,   0.   ],
       [  8.   , 183.   ,  64.   , ...,   0.672,  32.   ,   1.   ],
       ...,
       [  5.   , 121.   ,  72.   , ...,   0.245,  30.   ,   0.   ],
       [  1.   , 126.   ,  60.   , ...,   0.349,  47.   ,   1.   ],
       [  1.   ,  93.   ,  70.   , ...,   0.315,  23.   ,   0.   ]])

In [3]:
# split into input (X) and output (y) variables
X = dataset[:,0:8]
y = dataset[:,8]

## Defining the Model

Models in Keras are a sequence of layers. We create a `Sequential` model and add layers one at a time until we are happy with our network topology. The first thing to get right is to ensure the input layer has the right number of inputs. This can be specified when creating the first layer with the `input_dim` argument and setting it to `8` for the 8 input variables. 

How do we know the number of layers to use and their types? Trial and error experimentation. Generally, you need a network large enough to capture the structure of the problem. In this example, we will use a fully-connected network structure with three layers.

Fully connected layers are defined using the `Dense` class. We can specify the number f neurons in the layer as the first argument and specify the activation function using the `activation` argument. We will use the rectifier (`relu`) activation function on the first two layers and the `sigmoid` activation function in the output layer. It used to be the case that sigmoid and `tanh` activation functions were preferred for all layers. These days, better performance is seen using the rectifier activation function. For the output layer, we can snap the classification by setting a threshold, in this case 0.5.

The first hidden layer will have 12 neurons and expect 8 input variables (e.g. `input_dim=8`). The second hidden layer will have 8 neurons and finally the output layer will have 1 neuron to predict the class.

In [4]:
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

## Compiling the Model

The backend will automatically choose the best way to represent the network for training and making predictions to run on my hardware. When compiling, we must specify some additional properties required when training the network:
- The loss function to used to evaluate a set of weights
- The optimizer used to serach through different weights 
- Any optional metrics we would like to collect and report

In this case, we will use logarithmic loss, which for a binary classification problem is defined in Keras as `binary_crossentropy`. We will use the gradient descent algorithm `adam` as it is an efficient default. For metrics, we will capture and report the model accuracy.

In [5]:
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

## Fit the Model

We have defined our model and compiled it for efficient computation. Now it is time to execute the model on some data. We can train on our loaded dat by calling the `fit()` function on the model.

The training process will run for a fixed number of iterations through the dataset called epochs, that we must specify with the `epochs` argument. We can also set the number of instances that are evaluated before a weight update in the network is performed called the batch sixe and set using the `batch_size` argument. For this problem we will run a small number of epochs (150) and use a relatively small batch size of 16. 

In [6]:
# fit the model
model.fit(X, y, epochs=150, batch_size=16, verbose=0)

## Evaluate the Model

While we have an accuracy rating, it only applies to the training dataset. We do not have an out of sample dataset to work with. 

We can use the `evaluate()` function and pass it the same input/output used to train the model. This will generate a prediction for each input and output pair and collect scores, including the average loss and any metrics you have configured, such as accuracy.

In [7]:
# evaluate the keras model
_, accuracy = model.evaluate(X, y)

24/24 [==============================] - 0s 840us/step - loss: 0.4846 - accuracy: 0.7773
Accuracy: 77.73


Neural networks are a stochastic algorithm, meaning that the same algorithm on the same data can train a different model with different skill each time the code is run. This is a feature, not a bug. The variance in the performance of the model means that to get a reasomable approximation of how well your model is performing, you may need to fit it many times and calculate the average of the accuracy scores. 

In [8]:
def multiple_model_example(X=X, y=y, num=5):
    accuracies = []
    for i in range(num):
        model = Sequential()
        model.add(Dense(12, input_dim=8, activation='relu'))
        model.add(Dense(8, activation='relu'))
        model.add(Dense(1, activation='sigmoid'))
        # compile the keras model
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) # fit the keras model on the dataset
        model.fit(X, y, epochs=150, batch_size=16, verbose=0)
        # evaluate the keras model
        _, accuracy = model.evaluate(X, y)
        accuracies.append(accuracy*100)
    print("\n", f"The average accuracy over {num} fits is: {round (sum(accuracies) / len(accuracies), 2)}%")

In [9]:
multiple_model_example()

24/24 [==============================] - 0s 900us/step - loss: 0.4883 - accuracy: 0.7773

 The average accuracy over 5 fits is: 75.23%


## Make Predictions

We can adapt the above example and use it to generate predictions on the training dataset, pretending it is a new dataset we have not seen before. Making predictions is as easy as calling the `predict()` function on the model. We are using a sigmoid activation function on the output layer, so the predictions will be a probability in the range between 0 and 1. We can easily convert them into a crisp binary prediction for this classification task by rounding them.

In [10]:
# make probability predictions with the model
predictions = model.predict(X)

# round predictions
rounded = [round(x[0]) for x in predictions]

24/24 [==============================] - 0s 698us/step


In [11]:
print("Showing the first 5 predictions:")
print(rounded[:5])
print(f"There are {len(rounded)} predictions made for this set.")

Showing the first 5 predictions:
[1, 0, 1, 0, 1]
There are 768 predictions made for this set.


We can compare the predictions to the actual values in the target variable:

In [12]:
for i in range(5):
    print("%s => %d (expected %d)" % (X[i].tolist(), rounded[i], y[i]))

[6.0, 148.0, 72.0, 35.0, 0.0, 33.6, 0.627, 50.0] => 1 (expected 1)
[1.0, 85.0, 66.0, 29.0, 0.0, 26.6, 0.351, 31.0] => 0 (expected 0)
[8.0, 183.0, 64.0, 0.0, 0.0, 23.3, 0.672, 32.0] => 1 (expected 1)
[1.0, 89.0, 66.0, 23.0, 94.0, 28.1, 0.167, 21.0] => 0 (expected 0)
[0.0, 137.0, 40.0, 35.0, 168.0, 43.1, 2.288, 33.0] => 1 (expected 1)


## Considerations on Model Evaluation

There are a lot of decisions to make when designing and configuring deep learning models. Most of these decisions must be resolved empirically through trial and error. As such, it is critically important to have a robust way to evaluate the performance of a neural network. The following three methods will be explored: 
- Automatic verification
- Manual verification
- *k*-fold cross-validation

As many optimal topology structures have been developed through empirical evaluation, a simple approach is to begin by copying the structure of known optimal network topology and using heuristics. The best technique is to actually design small experiments and empirically evaluate options using real data. This includes high-level decisions like:
- Number of layers
- Size of layers
- Type of layers

And lower-level decisions like:
- Choice of loss function
- Choice of activation function
- Choice of optimization procedure
- Number of epochs

## Splitting

Large amounts of data combined with the complexity of neural networks result in exceptionally long training times. As such, it is typical to use a simple separation of data into training, validation, and test datasets. The use of splitting further reduces the risk of overfitting. 

### Using an Automatic Verification

Keras can separate a portion of the training data into a validation dataset and evalute the performance of the model on that validation dataset each epoch. This can be enabled by setting the `validation_split` argument on the `fit()` function to a percentage of the size of your training dataset. 

In [13]:
model.fit(X, y, validation_split=0.33, epochs=150, batch_size=10)

Epoch 1/150
52/52 [==============================] - 0s 4ms/step - loss: 0.5319 - accuracy: 0.7296 - val_loss: 0.5151 - val_accuracy: 0.7638
Epoch 2/150
52/52 [==============================] - 0s 2ms/step - loss: 0.5362 - accuracy: 0.7335 - val_loss: 0.5149 - val_accuracy: 0.7480
Epoch 3/150
52/52 [==============================] - 0s 2ms/step - loss: 0.5161 - accuracy: 0.7354 - val_loss: 0.4723 - val_accuracy: 0.8031
Epoch 4/150
52/52 [==============================] - 0s 2ms/step - loss: 0.5261 - accuracy: 0.7490 - val_loss: 0.4736 - val_accuracy: 0.8150
Epoch 5/150
52/52 [==============================] - 0s 2ms/step - loss: 0.5072 - accuracy: 0.7588 - val_loss: 0.4859 - val_accuracy: 0.8031
Epoch 6/150
52/52 [==============================] - 0s 2ms/step - loss: 0.5080 - accuracy: 0.7393 - val_loss: 0.4813 - val_accuracy: 0.8031
Epoch 7/150
52/52 [==============================] - 0s 2ms/step - loss: 0.5019 - accuracy: 0.7588 - val_loss: 0.4905 - val_accuracy: 0.8031
Epoch 8/150
5

Epoch 59/150
52/52 [==============================] - 0s 2ms/step - loss: 0.4870 - accuracy: 0.7529 - val_loss: 0.5206 - val_accuracy: 0.7756
Epoch 60/150
52/52 [==============================] - 0s 2ms/step - loss: 0.4830 - accuracy: 0.7646 - val_loss: 0.5136 - val_accuracy: 0.7756
Epoch 61/150
52/52 [==============================] - 0s 2ms/step - loss: 0.4802 - accuracy: 0.7588 - val_loss: 0.5237 - val_accuracy: 0.7638
Epoch 62/150
52/52 [==============================] - 0s 2ms/step - loss: 0.4802 - accuracy: 0.7529 - val_loss: 0.5117 - val_accuracy: 0.7953
Epoch 63/150
52/52 [==============================] - 0s 2ms/step - loss: 0.4905 - accuracy: 0.7665 - val_loss: 0.5185 - val_accuracy: 0.7835
Epoch 64/150
52/52 [==============================] - 0s 2ms/step - loss: 0.4889 - accuracy: 0.7393 - val_loss: 0.5970 - val_accuracy: 0.7283
Epoch 65/150
52/52 [==============================] - 0s 2ms/step - loss: 0.4992 - accuracy: 0.7529 - val_loss: 0.5036 - val_accuracy: 0.7598
Epoch 

52/52 [==============================] - 0s 2ms/step - loss: 0.4636 - accuracy: 0.7626 - val_loss: 0.5225 - val_accuracy: 0.7795
Epoch 117/150
52/52 [==============================] - 0s 2ms/step - loss: 0.4655 - accuracy: 0.7704 - val_loss: 0.5367 - val_accuracy: 0.7323
Epoch 118/150
52/52 [==============================] - 0s 2ms/step - loss: 0.4703 - accuracy: 0.7685 - val_loss: 0.5163 - val_accuracy: 0.7756
Epoch 119/150
52/52 [==============================] - 0s 2ms/step - loss: 0.4640 - accuracy: 0.7763 - val_loss: 0.5161 - val_accuracy: 0.7677
Epoch 120/150
52/52 [==============================] - 0s 2ms/step - loss: 0.4647 - accuracy: 0.7588 - val_loss: 0.5491 - val_accuracy: 0.7638
Epoch 121/150
52/52 [==============================] - 0s 2ms/step - loss: 0.4687 - accuracy: 0.7743 - val_loss: 0.5117 - val_accuracy: 0.7677
Epoch 122/150
52/52 [==============================] - 0s 2ms/step - loss: 0.4791 - accuracy: 0.7646 - val_loss: 0.5163 - val_accuracy: 0.7677
Epoch 123/150

### Using Manual Verification

We can use the `train_test_split()` function from scikit-learn to separate our data into a training and validation dataset. The validation dataset can be specified to the `fit()` function in Keras by the `validation_data` argument.

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=150, batch_size=10)

Epoch 1/150
52/52 [==============================] - 0s 2ms/step - loss: 0.5168 - accuracy: 0.7840 - val_loss: 0.4517 - val_accuracy: 0.7756
Epoch 2/150
52/52 [==============================] - 0s 2ms/step - loss: 0.5070 - accuracy: 0.7646 - val_loss: 0.4403 - val_accuracy: 0.7638
Epoch 3/150
52/52 [==============================] - 0s 2ms/step - loss: 0.5122 - accuracy: 0.7471 - val_loss: 0.4358 - val_accuracy: 0.7756
Epoch 4/150
52/52 [==============================] - 0s 1ms/step - loss: 0.5042 - accuracy: 0.7665 - val_loss: 0.4292 - val_accuracy: 0.7874
Epoch 5/150
52/52 [==============================] - 0s 1ms/step - loss: 0.5036 - accuracy: 0.7451 - val_loss: 0.4586 - val_accuracy: 0.7638
Epoch 6/150
52/52 [==============================] - 0s 1ms/step - loss: 0.4841 - accuracy: 0.7782 - val_loss: 0.4485 - val_accuracy: 0.7756
Epoch 7/150
52/52 [==============================] - 0s 1ms/step - loss: 0.4907 - accuracy: 0.7763 - val_loss: 0.4419 - val_accuracy: 0.7756
Epoch 8/150
5

Epoch 59/150
52/52 [==============================] - 0s 3ms/step - loss: 0.4656 - accuracy: 0.7782 - val_loss: 0.5258 - val_accuracy: 0.7402
Epoch 60/150
52/52 [==============================] - 0s 2ms/step - loss: 0.4667 - accuracy: 0.7957 - val_loss: 0.5245 - val_accuracy: 0.7598
Epoch 61/150
52/52 [==============================] - 0s 2ms/step - loss: 0.4739 - accuracy: 0.7685 - val_loss: 0.5091 - val_accuracy: 0.7559
Epoch 62/150
52/52 [==============================] - 0s 2ms/step - loss: 0.4691 - accuracy: 0.7704 - val_loss: 0.5077 - val_accuracy: 0.7677
Epoch 63/150
52/52 [==============================] - 0s 2ms/step - loss: 0.4639 - accuracy: 0.7918 - val_loss: 0.5396 - val_accuracy: 0.7559
Epoch 64/150
52/52 [==============================] - 0s 2ms/step - loss: 0.4804 - accuracy: 0.7626 - val_loss: 0.5413 - val_accuracy: 0.7480
Epoch 65/150
52/52 [==============================] - 0s 2ms/step - loss: 0.4784 - accuracy: 0.7782 - val_loss: 0.5358 - val_accuracy: 0.7441
Epoch 

52/52 [==============================] - 0s 1ms/step - loss: 0.4604 - accuracy: 0.7743 - val_loss: 0.5709 - val_accuracy: 0.7520
Epoch 117/150
52/52 [==============================] - 0s 1ms/step - loss: 0.4521 - accuracy: 0.7763 - val_loss: 0.6112 - val_accuracy: 0.7520
Epoch 118/150
52/52 [==============================] - 0s 1ms/step - loss: 0.4614 - accuracy: 0.7763 - val_loss: 0.5712 - val_accuracy: 0.7795
Epoch 119/150
52/52 [==============================] - 0s 1ms/step - loss: 0.4709 - accuracy: 0.7685 - val_loss: 0.5655 - val_accuracy: 0.7441
Epoch 120/150
52/52 [==============================] - 0s 1ms/step - loss: 0.4529 - accuracy: 0.7840 - val_loss: 0.5769 - val_accuracy: 0.7598
Epoch 121/150
52/52 [==============================] - 0s 1ms/step - loss: 0.4608 - accuracy: 0.7899 - val_loss: 0.5672 - val_accuracy: 0.7520
Epoch 122/150
52/52 [==============================] - 0s 1ms/step - loss: 0.4542 - accuracy: 0.8074 - val_loss: 0.5800 - val_accuracy: 0.7480
Epoch 123/150

### Manual *k*-Fold Cross-Validation

The gold standard for machine learning model evaluation in *k*-fold cross-validation. It provides a robust estimate of the performance of a model on unseen data. It does this by splitting the training dataset into *k* subsets and takes turns training models on all subsets except one which is held out, and evaluating model performance on the held out validation dataset. The process is repeated until all subsets are given an opportunity to be the held out validation set. The performance measure is then averaged across all models that are created. 

Cross-validation is often **not** used for evaluating deep learning models because of the greater computational expense. For example *k*-fold cross-validation is often used with 5 or 10 folds. As such, 5 or 10 models must be constructed and evaluated, greatly adding to the evaluation time of a model. Nevertheless, when the problem is small enough or if there are sufficient compute resources, *k*-fold cross-validation can give a less biased estimate of the peformance of the model. 

We can use the `StratifiedKFold` class from scikit-learn to split the training dataset into 10 folds. The folds are straified, meaning that the algorithm attempts to balance the number of instances of each class in each fold. The example creates and evaluates 10 models using the 10 splits of the data and collects all of the scores. The average and standard deviation of the model performance is then printed at the end of the run to provide a robust estimate of model accuracy.

In [15]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
# define 10-fold cross validation test harness
kfold = StratifiedKFold(n_splits=10, shuffle=True)
cvscores = []

# fitting models on each split (we will use the existing model already defined and compiled)
for train, test in kfold.split(X, y):
    # fit the model
    model.fit(X[train], y[train], epochs=150, batch_size=10, verbose=0)
    # evaluate the model
    scores = model.evaluate(X[test], y[test], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)

In [17]:
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

77.48% (+/- 6.84%)


## Leveraging Scikit-Learn

The scikit-learn library is the most popular library for general machine learning in Python. We can use scikit-learn as a wrapper for Keras to employ the familiar functions and methods used in machine learning development. The Keras library provides a convenient wrapper for deep learning models to be used as classification or regression estimators in scikit-learn. We will work through examples of using the `KerasClassifier` wrapper for a classification neural network created in Keras and used in the scikit-learn library. Relatively recently, the keras.wrappers.scikit_learn library has been deprecated in favor of the scikeras library. We will use the latter. 

In [18]:
#! pip install scikeras

The `KerasClassifier` and `KerasRegressor` classes in Keras take an argument `build_fn` which is the name of the function to call to create the model. We must define a function that defines the model, compiles it, and returns it. In the example to follow, we will define a function called `create_model()` which will perform these steps for us.

We can then pass this function name to the `KerasClassifier` class by the `build_fn` argument. We also pass in additional arguments of `epochs=150` and `batch_size=10`. These are automatically bundled up and passed on to the `fit()` function which is called internally by the `KerasClassifier` class. We then can use the scikit-learn function `cross_val_score()` to evaluate our model. 

In [19]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score

In [20]:
# Function to create model, required for KerasClassifier
def create_model():
    # define model
    model = Sequential()
    model.add(Dense(12, input_dim=8, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [21]:
# Create model using KerasClassifier
model = KerasClassifier(build_fn=create_model, epochs=150, batch_size=10, verbose=0)

> Note: In a future relase, the `build_fn` argument will be deprecated. The argument `model` will take its place.

In [22]:
# Evaluate using 10-fold cross validation
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(model, X, y, cv=kfold)
print(results.mean())

0.7122009569377991


When the Keras model is wrapped that estimating model accuracy can be greatly streamlined, compared to the manual enumeration of cross-validation folds.

## Grid Search of Deep Learning Model Parameters

We already know we can provide arguments to the `fit()` function. The function that we specify to the `build_fn/model` argument when creating the `KerasClassifier` wrapper can also take arguments. We can use these arguments to further customize the construction of the model. 

In this example, we use a grid search to evaluate different configurations for our neural network model and report on the combination that provides the best estimated performance. The `create_model()` function is defined to take two arguments `optimizer` and `init`, both of which must have default values. This will allow us to evaluate the effect of using different optimization algorithms and weight initialization schemes for our network. After creating our model, we define arrays of values for the parameter we wish to search, specifically:
- Optimizers for searching different weight values.
- Initializers for preparing the network weights using different schemes.
- Number of eopchs for training the model for different number of exposures to the training dataset.
- Batches for varying the number of samples before weight updates.

The options are specified into a dictionary and passed to the configuration of the `GridSearchCV` scikit-learn class. This class will evaluate a version of our nerual network model for each combination of parameters for the combinations of optimizers, initializations, epochs and batches. Each combination is then evaluated using the default of 3-fold stratified cross-validation. In this first example, we will only perform a grid search on the initializers. 

> Note: It may be useful to design small experiments with a smaller subset of the data, as this process can be incredibly compute and time intensive. 

Finally, the performance and combination of configurations for the best model are displayed, followed by the performance of all combinations of parameters.

In [23]:
from sklearn.model_selection import GridSearchCV

In [24]:
# Function to create model, required for KerasClassifier with optimizer and init argument values passed
def create_model(init_mode='uniform'):
    # create model
    model = Sequential()
    model.add(Dense(12, input_shape=(8,), kernel_initializer=init_mode, activation='relu'))
    model.add(Dense(1, kernel_initializer=init_mode, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [26]:
# create model
model = KerasClassifier(model=create_model, epochs=100, batch_size=10, verbose=0)
# define the grid search parameters
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
param_grid = dict(model__init_mode=init_mode)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, y)

In [27]:
# Summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.721354 using {'model__init_mode': 'normal'}
0.708333 (0.028940) with: {'model__init_mode': 'uniform'}
0.671875 (0.026107) with: {'model__init_mode': 'lecun_uniform'}
0.721354 (0.033804) with: {'model__init_mode': 'normal'}
0.651042 (0.001841) with: {'model__init_mode': 'zero'}
0.704427 (0.037783) with: {'model__init_mode': 'glorot_normal'}
0.694010 (0.016367) with: {'model__init_mode': 'glorot_uniform'}
0.695312 (0.019401) with: {'model__init_mode': 'he_normal'}
0.666667 (0.027126) with: {'model__init_mode': 'he_uniform'}


Based on the results of the experiment, utilizing the `normal` initializer produces the best results. We can increase the complexity of the grid search by including additional elements, but the computational demands of this process increase substantially when doing so. 

In [28]:
# Define the grid search parameters
batch_size = [5, 10, 20]
epochs = [50, 100, 150]
init_mode = ['uniform', 'normal', 'zero']
optimizer = ['RMSprop', 'Adam', 'SGD']

# Create the dictionary for the param_grid argument
param_grid = dict(optimizer=optimizer, 
                  batch_size=batch_size, 
                  epochs=epochs, 
                  model__init_mode=init_mode)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, y)

In [29]:
# Summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.734375 using {'batch_size': 10, 'epochs': 100, 'model__init_mode': 'uniform', 'optimizer': 'Adam'}
0.707031 (0.026107) with: {'batch_size': 5, 'epochs': 50, 'model__init_mode': 'uniform', 'optimizer': 'RMSprop'}
0.714844 (0.011500) with: {'batch_size': 5, 'epochs': 50, 'model__init_mode': 'uniform', 'optimizer': 'Adam'}
0.692708 (0.021236) with: {'batch_size': 5, 'epochs': 50, 'model__init_mode': 'uniform', 'optimizer': 'SGD'}
0.701823 (0.025976) with: {'batch_size': 5, 'epochs': 50, 'model__init_mode': 'normal', 'optimizer': 'RMSprop'}
0.717448 (0.034104) with: {'batch_size': 5, 'epochs': 50, 'model__init_mode': 'normal', 'optimizer': 'Adam'}
0.701823 (0.016053) with: {'batch_size': 5, 'epochs': 50, 'model__init_mode': 'normal', 'optimizer': 'SGD'}
0.651042 (0.001841) with: {'batch_size': 5, 'epochs': 50, 'model__init_mode': 'zero', 'optimizer': 'RMSprop'}
0.651042 (0.001841) with: {'batch_size': 5, 'epochs': 50, 'model__init_mode': 'zero', 'optimizer': 'Adam'}
0.651042 (0.001

In the example, we have identified the optimal hyperparameter setup from the options given to the grid search algorithm. Additional considerations can be given to:
- Learning Rate: How much to update the weight at the end of each batch.
- Momentum: How much to let the previous update influence the current update.
- Neuron Activation Function: Control of the non-linearity of individual neurons and when to fire.
- Dropout Regularization: Limit overfitting by randomly dropping out nodes during training.


## Helpful Tips:
- Carefully choose cross validation configuration to ensure stable results
- Do not just focus on the best result, review the whole grid of results and look for trends among hyperparameter tunings
- Parallelize training to draw on all compute resources available
- Use a smaller sample of the data to speed up grid search investigations
- Start with coarse grids and fine tune once the general direction of optimal parameterization is identified
- Recognize that hyperparameter tuning in neural networks is extremely dependent on the data (do not generalize settings to different problems)
- Recognize that reproducibility in neural networks is inherently difficult